# Deep learning based diffusive size factors

A connected pore-throat-pore element in the PNM is defined as a conduit and the estimation of conduit’s conductance for mass transports has been an active research area since the PNM technique was developed. Common shape factor approach in estimating diffusive conductance can be erroneous for real void space with highly complex geometrical shapes. An alternative to analytical methods based on shape factor is to use DNS to estimate the flow/diffusive conductance of pore network elements. Although this approach increases the PNM accuracy, it is computationally expensive to conduct this analysis on every tomogram that the pore network is extracted. This example demonstrates how PoreSpy tackles this problem to estimate diffusive size factors of conduits using a deep learning framework.

The deep learning framework takes in only images of segmented porous medium and returns an array of diffusive size factors for all conduits in the image. The framework uses convolutional neural networks to extract the features of the image.

The framework can be applied to both one conduit image as well as a segmented image of porous medium:

<img src="https://user-images.githubusercontent.com/43128873/116145783-bd458380-a6ab-11eb-8103-fb9a93f0599c.png" alt="PS_dl" width="500"/>

# Trained model and supplementary materials
The deep learning model is developed to estimate diffusive size factors of conduits in a porous medium (currently trained 
on polydispersed spheres). To enable this feature, please download the trained model or weight along with trained data distribution. The trained model and this example's materials are downloadable as following:

[trained model](link): This file includes both weights and architecture of the deep learning layers. To use this file, there is no need to build a model

[trained model weights only](link): This file includes only weights of the deep learning layers. To use this file, a Resnet50 model (provided in PoreSpAI) must be built beforehand.

[trained data distribution](link): This file will be used in denormalizing predicted values based on normalized transform applied on training data.

[Example materials](link): Materials for this example (samples and results) 

## Import the packages

In [4]:
import scipy as sp
import porespy as ps
import numpy as np
import h5py
import gc
from tqdm import tqdm
from matplotlib import pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")
# path to the downloaded supplementary materials, by default current directory is used here:
path = os.getcwd()+'/'

## Create images for testing

We have created a 3D image using PoreSpy's `poly_disperese_spheres` generator and segmented using `snow_partitioning methods`.

In [ ]:
#create im code here
regions_1=...

Let's try the method for a segmented image. The user can choose between two methods of predicting the diffusive size factors. 

1) The value for each conduit will be calculated separately in a for loop

2) The values for all conduits will be calculated once using a tensor

The second method is faster, as tensorflow deep learning model is designed to work with tensors. The difference in the speed is more as the number of conduit images in segmented regions increases.

### A segmented region image

The segmented region is a $200^3$ image of a random polydisperese sphere packs. First, to show the speed difference between tensor-wise prediction and individual predictions, we use part of this image a $50^3$ segmented region.


<img src="https://user-images.githubusercontent.com/43128873/115746110-0e7b0d80-a362-11eb-90c3-276d2fceb4d6.png" alt="PS_dl" width="300"/>

In [ ]:
import timeit
start = timeit.default_timer()
g=ps.networks.AI_size_factor(regions_1[:50,:50,:50], tensor_wise=True, model_dir=path+'saved_model.h5', build_model=False)
stop = timeit.default_timer()
print('Time tensor-wise=True: ', stop - start)
start = timeit.default_timer()
g=ps.networks.AI_size_factor(regions_1[:50,:50,:50], tensor_wise=False,  model_dir=path+'saved_model.h5', build_model=False)
stop = timeit.default_timer()
print('Time tensor-wise=False: ', stop - start)

Note: This runtime can be reduced by running tensorflow model on GPU instead of CPU. To use tensorflow models on GPU, tensorflow-gpu is required. For a larger image, the difference between tensor-wise and individual-pair predictions will be higher. Note: For a larger image, the difference between tensor-wise and individual-pair predictions will be higher. A larger part of this reported time is related to extracting the pairs of conduits (which is the common step required for both AI and finite difference method). Once the data is prepared, AI Prediction on the tensor takes a small amount of the runtime.